In [1]:
# import libs and env
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
import dotenv
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
from termcolor import colored  
import wget
import os
import chromadb
from ast import literal_eval
#GPT_MODEL = "gpt-4-1106-preview"
GPT_MODEL = "gpt-3.5-turbo-0125"
EMBEDDING_MODEL = "text-embedding-ada-002"
load_dotenv()  

True

### **utilities**

In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


#### **chroma**

In [4]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

type(response.data[0].embedding)

list

In [6]:
# Placeholder function for generating embeddings using OpenAI
def generate_embedding(text):
    """
    Generates an embedding vector for the given text using OpenAI's embedding API.
    Replace this with an actual call to openai.Embedding.create() in your code.
    """
    response = client.embeddings.create(
        input=text,
        model=EMBEDDING_MODEL
    )

    return response.data[0].embedding  # Assuming embedding size is 768

def process_md_files_to_df(folder_path):
    """
    Processes all markdown files in the given folder and compiles them into a DataFrame.
    
    :param folder_path: Path to the folder containing MD files.
    :return: A DataFrame with the columns id, file_path, title, text, title_vector, content_vector, vector_id.
    """
    data = []
    
    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith(".md"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                title = os.path.splitext(filename)[0]
                title_vector = generate_embedding(title)
                content_vector = generate_embedding(content)
                
                # Append to data list
                data.append({
                    "id": idx + 1,
                    "file_path": file_path,
                    "title": title,
                    "text": content,
                    "title_vector": title_vector,
                    "content_vector": content_vector,
                    "vector_id": idx  # Assuming vector_id starts from 0
                })
    
    # Convert list to DataFrame
    df = pd.DataFrame(data)
    return df

# Example usage
folder_path = "sop-docs/euroclear"

In [7]:
article_df = process_md_files_to_df(folder_path)

In [8]:
article_df.to_csv('sop-docs/oa-sop-docs.csv', index=False)

In [9]:
article_df.head()

,id,file_path,title,text,title_vector,content_vector,vector_id
0,1,sop-docs/euroclear/external-settlement.md,external-settlement,## External settlement\n19/09/2022\n\n### What...,"[-0.0016799656441435218, 0.0029202322475612164...","[-0.001593194087035954, 0.02452816814184189, -...",0
1,2,sop-docs/euroclear/transaction-lifecycle.md,transaction-lifecycle,## What is the lifecycle of transactions?\n\nT...,"[0.004535823594778776, -0.004832559730857611, ...","[-0.018600638955831528, 0.0048954663798213005,...",1
2,3,sop-docs/euroclear/bridge-settlement.md,bridge-settlement,## Bridge settlement\n\n### What is a Bridge s...,"[-0.004899136256426573, 0.007877608761191368, ...","[-0.012175136245787144, 0.008481089025735855, ...",2
3,4,sop-docs/euroclear/status-reporting.md,status-reporting,"## Unmatched, unsettled, alleged reporting\n21...","[-0.024325305595993996, -0.008656425401568413,...","[-0.023916499689221382, 0.01272289827466011, 0...",3
4,5,sop-docs/euroclear/internal-settlement.md,internal-settlement,## What is an internal settlement transaction?...,"[-0.010655367746949196, 0.009483421221375465, ...","[0.008218420669436455, 0.0009066986385732889, ...",4


In [10]:
type(article_df.title_vector[0])

list

In [11]:
# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

In [12]:
chroma_client = chromadb.EphemeralClient() # Equivalent to chromadb.Client(), ephemeral.
# Uncomment for persistent client
# chroma_client = chromadb.PersistentClient()

In [13]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name=EMBEDDING_MODEL)

sop_content_collection = chroma_client.create_collection(name='sop_contents_v4', embedding_function=embedding_function)
sop_title_collection = chroma_client.create_collection(name='sop_titles_v4', embedding_function=embedding_function)

# Add the content vectors
sop_content_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.content_vector.tolist(),
)

# Add the title vectors
sop_title_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.title_vector.tolist(),
)

In [15]:
def search_df(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'title': dataframe[dataframe.vector_id.isin(results['ids'][0])]['title'],
                'content': dataframe[dataframe.vector_id.isin(results['ids'][0])]['text'],
                })
    
    return df

In [16]:
title_query_result = search_df(
    collection=sop_title_collection,
    query="serbia tourism",
    max_results=1,
    dataframe=article_df
)
title_query_result.head()

,id,score,title,content
0,0,0.13953,serbia,# Serbia\n\n![Serbia Landscape](URL_to_an_imag...


In [17]:
content_query_result = search_df(
    collection=sop_content_collection,
    query="serbia tourism",
    max_results=1,
    dataframe=article_df
)
content_query_result.head()

,id,score,title,content
0,0,0.251935,serbia,# Serbia\n\n![Serbia Landscape](URL_to_an_imag...


In [ ]:
# Function to create the desired string from a DataFrame
def search_knowledge(query, articles, n, clearing_house="na", market="na"):
    df = search_df(
    collection=sop_title_collection,
    query=query,
    max_results=n,
    dataframe=articles
    )
    # Initialize an empty string
    result_string = ""
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Append the title and content to the result string, followed by a newline character
        result_string += f"{row['title']}\n{row['content']}\n"
    
    return result_string.strip()  # Use strip() to remove the last newline character

#### **chat completion wrapper**

In [ ]:
# wrapper around chat api call
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    client = OpenAI()
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e
    
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request_json(messages, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            response_format={ "type": "json_object" },
            messages=messages
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

#### **agent functions**

In [82]:
# tools
agent_functions = [
{
    "type": "function",
    "function": {
        "name": "search_knowledge",
        "description": "search different knowledge bases based on user query about securities operations, also has info about Polanski and serbia",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query",
                },
                "clearing_house": {
                    "type": "string",
                    "enum": ["euroclear", "cmu", "austraclear", "dtc", "na"],
                    "description": "The clearing house that the user is interested in",
                },
                    "market": {
                    "type": "string",
                    "enum": ["united_states", "australia", "china", "na"],
                    "description": "The market that the user is interested in",
                },
            },
            "required": ["query", "clearing_house", "market"],
        },
    }
},
{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
},
]


#### **agent personas**

In [83]:
# agent persona
GENERAL_ASSISTANT = """
You are Sage, a friendly and helpful agent who helps users using the given functions.
Here are the rules you have to strictly adhere to:
1. Don't make assumptions about what values to plug into functions. 
2. You will use the search tool to find relavent knowlege articles to create the answer.
3. Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
4. Review the result of the search and use it to guide your next search if needed.
5. If the question is complex, break down to smaller search steps and find the answer in multiple steps.
6. Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
"""

RESEARCHER = """
You are ResearchGPT, a helpful assistant who uses a given list of sources to take notes in order to answer a question. Your notes
would be passed to a question generator which would generate additional questions to answer the question in full if there is not
enough information in the notepad. 
Hints:
1. Do not comment, just diligently take notes
2. Extract a list of all the facts directly related to the question concisely onto your notepad.
"""

QUESTION_GENERATOR = """
You are QuestionsGPT, a helpful assistant who looks at the current state of acquired knowledge
and intelligently generates an additional question and query to search a knowledge base.
Here are some tips;
1. Create questions and queries which can concisely and dircetly answer the user's primary query
Provide your answer in JSON structure like this {"enough_info": "is there enough info in the notepad to anser this question. allowed values: yes/no", "additional_query": "your search query to get additional info to answer user question if not enough info else 'na'"}
"""

FINAL_ANSWERER = """
Using the given information in a notepad from a researcher, concisely answer the user question.
"""

#### **agent functions**

In [84]:
def generate_qns(query, notepad):
    notepad_example = """
    - Polanski was born in Serbia and started playing tennis at 4 years old.
    - Turned professional at age 16 after an impressive junior career.
    - His playing style includes offensive baseline play and quick net skills, known for a signature serve.
    - Has reached semifinals of several ATP 250 events and has victories over higher-ranked opponents.
    - Polanski runs tennis clinics for underprivileged children in his home country.
    - Experts predict he will continue to rise in the ranks of professional tennis.
    """
    example_query1 = "where was Polanski born?"
    example_query2 = "what is the capital of the place where Polanski was born?"
    messages = []
    messages.append({"role": "system", "content": f"{QUESTION_GENERATOR}"})
    messages.append({"role": "user", "content": f"Is this enough to answer the user's initial question: {example_query1}\n Here's the notepad: {notepad_example}\n What additional question has to generated?"})
    messages.append({"role": "assistant", "content": '{"enough_info": "yes", "additional_query": "na"}'})
    messages.append({"role": "user", "content": f"Is this enough to answer the user's initial question: {example_query2}\n Here's the notepad: {notepad_example}\n What additional question has to generated?"})
    messages.append({"role": "assistant", "content": '{"enough_info": "no", "additional_query": "current capital of Serbia"}'})
    messages.append({"role": "user", "content": f"Is this enough to answer the user's initial question: {query}\n Here's the notepad we have: {notepad}\n What additional question has to generated?"}) 
    chat_response = chat_completion_request_json(
        messages
    )
    # generate json response
    assistant_message = chat_response.choices[0].message.content
    return assistant_message

In [85]:
def final_answer(query, notepad):
    example_query1 = "where was Polanski born?"
    example_query2 = "what is the capital of the place where Polanski was born?"
    messages = []
    messages.append({"role": "system", "content": f"{FINAL_ANSWERER}"})
    messages.append({"role": "user", "content": f"query: {query}\n Here's the notepad we have for your reference: {notepad}"}) 
    chat_response = chat_completion_request(
        messages
    )
    # generate json response
    assistant_message = chat_response.choices[0].message.content
    return assistant_message

In [86]:
def get_current_weather(location, format):
    out = f"weather at {location} is 22 degrees C!"
    return out


def call_researcher(query, articles, n):
    notepad = ""
    user_query = query
    enough_info = 'no'
    i = 0

    while enough_info.lower() == 'no':
        messages=[]
        search_results = search_knowledge(query, articles, n) 
        #print(search_results)
        if not messages:
            messages.append({"role": "system", "content": f"{RESEARCHER}"})

        messages.append({"role": "user", "content": f":these are your sources {search_results} \nExtract only the relevant information to this query: {query}\n into your notepad."})
        
        chat_response = chat_completion_request(
            messages
        )
        assistant_message = chat_response.choices[0].message.content
        notepad += assistant_message
        print(f"\n{notepad}")
        generated_qns = generate_qns(user_query, notepad)
        additional_qns = json.loads(generated_qns)
        enough_info = additional_qns['enough_info']
        query = additional_qns['additional_query']
        print(enough_info)
        print(query)
        i += 1
        if i == 3:
            break

    answer = final_answer(user_query, notepad)
    return answer

AVAILABLE_FUNCTIONS = {
            "search_knowledge": search_knowledge,
            "get_current_weather": get_current_weather
        } 

function_list=AVAILABLE_FUNCTIONS

In [87]:
#answer = call_researcher("what is the capital of the country which seperated from the place where Polanski was born?", article_df, n=1)

In [88]:
Markdown(answer)

NameError: name 'answer' is not defined

In [89]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

#### **function calling**

In [90]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    while True:
        response = chat_completion_request(messages, functions)
        full_message = response.choices[0]
        print(full_message)
    
        if full_message.finish_reason == "tool_calls":
            print(f"Function generation requested, calling function")
            call_agent_function(messages, full_message)
        else:
            print(f"Function not required, responding to user")
            return response
            


def call_agent_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    if full_message.message.tool_calls[0].function.name == "search_knowledge":
        try:
            parsed_output = json.loads(
                full_message.message.tool_calls[0].function.arguments
            )
            print("Getting search results")
            results = search_knowledge(parsed_output["query"], articles=article_df, n=1)
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        messages.append(
            {
                "role": "function",
                "name": full_message.message.tool_calls[0].function.name,
                "content": str(results),
            }
        )
        try:
            print("Got search results, summarizing content")
            response = chat_completion_request(messages)
            return response
        except Exception as e:
            print(type(e))
            raise Exception("Function chat request failed")
    elif (
        full_message.message.tool_calls[0].function.name == "get_current_weather"
    ):
        parsed_output = json.loads(
            full_message.message.tool_calls[0].function.arguments
        )
        print("Finding and reading paper")
        summary = get_current_weather(parsed_output["location"], parsed_output["format"])
        return summary

    else:
        raise Exception("Function does not exist and cannot be called")


### **demo**

In [94]:
# system message
paper_conversation = Conversation()
paper_conversation.add_message("system", GENERAL_ASSISTANT)

In [95]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': "\nYou are Sage, a friendly and helpful agent who helps users using the given functions.\nHere are the rules you have to strictly adhere to:\n1. Don't make assumptions about what values to plug into functions. \n2. You will use the search tool to find relavent knowlege articles to create the answer.\n3. Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.\n4. Review the result of the search and use it to guide your next search if needed.\n5. If the question is complex, break down to smaller search steps and find the answer in multiple steps.\n6. Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\n"}]

In [96]:
# user message
paper_conversation.add_message("user", "what is a tourism webiste of the place tennis player polanski was born?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=agent_functions
)
assistant_message = chat_response.choices[0].message.content
#assistant_message = chat_response
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1VFWIuQYw3ZNmDwZqG4LnNtf', function=Function(arguments='{"query":"tennis player Polanski birthplace","clearing_house":"na","market":"na"}', name='search_knowledge'), type='function')]))
Function generation requested, calling function
Getting search results
Got search results, summarizing content
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_prBVkLrpcR8LGJVyvH05irCQ', function=Function(arguments='{"query":"Serbia tourism website","clearing_house":"na","market":"na"}', name='search_knowledge'), type='function')]))
Function generation requested, calling function
Getting search results
Got search results, summarizing content
Choice(finish_reason='stop', inde

The tourism website of Serbia, the place where tennis player Polanski was born, is [Serbia's official tourism website](http://www.serbia.travel).

In [97]:
pretty_print_conversation(paper_conversation.conversation_history)

system: 
You are Sage, a friendly and helpful agent who helps users using the given functions.
Here are the rules you have to strictly adhere to:
1. Don't make assumptions about what values to plug into functions. 
2. You will use the search tool to find relavent knowlege articles to create the answer.
3. Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
4. Review the result of the search and use it to guide your next search if needed.
5. If the question is complex, break down to smaller search steps and find the answer in multiple steps.
6. Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.


user: what is a tourism webiste of the place tennis player polanski was born?

function (search_knowledge): polanski
# Polanski: Serbia's Rising Te

In [964]:
# user message
paper_conversation.add_message("user", "i'm intersted in united states?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=agent_functions
)
assistant_message = chat_response.choices[0].message.content
#assistant_message = chat_response
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lJmYH2iZMTT0wpctNz7UhOw7', function=Function(arguments='{"query":"cross border transfer","clearing_house":"cmu","market":"united_states"}', name='search_knowledge'), type='function')]))
Function generation requested, calling function
Getting search results
Got search results, summarizing content


The knowledge base does not contain specific information on conducting cross-border transfers within the United States through the Central Moneymarkets Unit (CMU). Typically, cross-border transfers may involve different processes depending on the specific financial institutions and the regulatory requirements for cross-border transactions in the involved jurisdictions.

For detailed and accurate procedures regarding cross-border transfers within the CMU involving the United States, you would likely need to consult with the CMU directly or through a financial institution that facilitates such transfers, as they can provide specific guidance and instructions based on current practices and regulations.

### **old**

In [ ]:
messages = []
# takes user input, outputs resopnse, appends message to messages
def llm_chat(input, messages): 
    if not messages:
        messages.append({"role": "system", "content": f"{GENERAL_ASSISTANT}"})

    messages.append({"role": "user", "content": f"{input}"})
    chat_response = chat_completion_request(
        messages, tools=agent_functions
    )
    assistant_message = chat_response.choices[0].message
    #messages.append({"role": "system", "content": assistant_message})

    return assistant_message

In [ ]:
def function_calling(tool_calls):
    for tool_call in tool_calls:
        function_name = tool_call.function.name
    print("Recommended Function call:")
    print(function_name)
    print()

    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
                    
    # verify function exists
    function_to_call = function_list[function_name]
    
    # verify function has correct number of arguments
    function_args = json.loads(tool_call.function.arguments)
    
    # print("beginning function call")
    function_response = str(function_to_call(**function_args))

    return function_response

In [ ]:
# keep last n convo, keeping the system prompt
def smart_agent(user_input, messages, n):
    print(f"keeping last {n} convo")

    if not messages:
        messages.append({"role": "system", "content": f"{GENERAL_ASSISTANT}"})

    if user_input:
        messages.append({"role": "user", "content": f"{user_input}"})
        
    chat_response = chat_completion_request(
        messages, tools=agent_functions
    )
    assistant_message = chat_response.choices[0].message
    #response = llm_chat(user_input, messages)
    tool_calls = assistant_message.tool_calls
    chat_state = messages
    content = assistant_message.content
    
    if content:
        chat_state.append({"role":"system", "content": content})
    print(content)
    if tool_calls:
        function_out = function_calling(tool_calls)
        chat_state.append({"role": "system", "content": function_out})

    return chat_state
    

In [343]:
messages = []

In [344]:
chats = smart_agent("how to do a cross border security transfer", messages, n=3)

keeping last 3 convo
To perform a cross-border security transfer, you generally need to follow these steps:

1. **Contact Financial Intermediaries**: You usually need to work with financial intermediaries such as banks, brokers, or custodians that have operations in both the sending and receiving countries.

2. **Understand Regulations**: The regulations for security transfers vary by country and can include tax laws, anti-money laundering checks, and other compliance requirements. It's essential to understand the relevant regulations in both jurisdictions.

3. **Use of International Securities Identification Numbers (ISINs)**: ISINs are used to uniquely identify securities to ensure the correct transfer of ownership across borders.

4. **Choose a Settlement Method**: Decide on a method of settlement. Most cross-border transfers are performed through international clearing systems such as Euroclear, Clearstream, or local central securities depositories (CSDs).

5. **Initiate the Transf

In [345]:
chats

[{'role': 'system',
  'content': "\nYou are Sage, a friendly and helpful agent who helps users using the given functions.\nHere are your rules:\n1. Don't make assumptions about what values to plug into functions. \n2. Ask for clarification if a user request is ambiguous.\n3. For questions about trade operations and securities, always respond based on the knowledge base. Do not make up your own answer.\n"},
 {'role': 'user', 'content': 'how to do a cross border security transfer'},
 {'role': 'system',
  'content': "To perform a cross-border security transfer, you generally need to follow these steps:\n\n1. **Contact Financial Intermediaries**: You usually need to work with financial intermediaries such as banks, brokers, or custodians that have operations in both the sending and receiving countries.\n\n2. **Understand Regulations**: The regulations for security transfers vary by country and can include tax laws, anti-money laundering checks, and other compliance requirements. It's essen

In [346]:
chats_2 = smart_agent("I'm interested in euroclear and united states", chats, n=3)

keeping last 3 convo
None
Recommended Function call:
search_knowledge



In [347]:
chats_2

[{'role': 'system',
  'content': "\nYou are Sage, a friendly and helpful agent who helps users using the given functions.\nHere are your rules:\n1. Don't make assumptions about what values to plug into functions. \n2. Ask for clarification if a user request is ambiguous.\n3. For questions about trade operations and securities, always respond based on the knowledge base. Do not make up your own answer.\n"},
 {'role': 'user', 'content': 'how to do a cross border security transfer'},
 {'role': 'system',
  'content': "To perform a cross-border security transfer, you generally need to follow these steps:\n\n1. **Contact Financial Intermediaries**: You usually need to work with financial intermediaries such as banks, brokers, or custodians that have operations in both the sending and receiving countries.\n\n2. **Understand Regulations**: The regulations for security transfers vary by country and can include tax laws, anti-money laundering checks, and other compliance requirements. It's essen

In [348]:
chats_3 = smart_agent(user_input=None, messages=chats_2, n=3)

keeping last 3 convo
None
Recommended Function call:
search_knowledge



In [349]:
chats_3

[{'role': 'system',
  'content': "\nYou are Sage, a friendly and helpful agent who helps users using the given functions.\nHere are your rules:\n1. Don't make assumptions about what values to plug into functions. \n2. Ask for clarification if a user request is ambiguous.\n3. For questions about trade operations and securities, always respond based on the knowledge base. Do not make up your own answer.\n"},
 {'role': 'user', 'content': 'how to do a cross border security transfer'},
 {'role': 'system',
  'content': "To perform a cross-border security transfer, you generally need to follow these steps:\n\n1. **Contact Financial Intermediaries**: You usually need to work with financial intermediaries such as banks, brokers, or custodians that have operations in both the sending and receiving countries.\n\n2. **Understand Regulations**: The regulations for security transfers vary by country and can include tax laws, anti-money laundering checks, and other compliance requirements. It's essen

In [317]:
pretty_print_conversation(chats)

system: 
You are Sage, a friendly and helpful agent who helps users using the given functions.
Here are your rules:
1. Don't make assumptions about what values to plug into functions. 
2. Ask for clarification if a user request is ambiguous.
3. For questions about trade operations and securities, always respond based on the knowledge base. Do not make up your own answer.


user: how to do a cross border security transfer

system: To execute a cross-border security transfer, multiple factors must be considered, including the regulatory requirements of the jurisdictions involved, the types of securities being transferred, the clearing houses, and the specific procedures laid out by the intermediaries and financial institutions involved in the transfer.

Here's a general outline of the process:

1. **Initiation**: Contact your financial institution or broker to start the transfer process. You will need to provide details about the securities you wish to transfer, the quantity, and the des

In [ ]:
# add iterative functionality - recursive questioning using function calling
# in smart agent, he uses a while true on function calling - this could be added with some breaks